# Fossil fuel imports and exports to South Africa, Nigeria and Egypt
This notebook calculates the imports and exports of fossil fuels from named countries, using data from https://comtrade.un.org/data. We use the designation "from" to indicate data from the country we are considering, and "to" to reference data originating from other countries - this should not be confused with import and export data, which are found in both datasets. 

Note that references to US are using a conversion database that originated in the USA, not simply accounting for trade between the USA and the country in question. 

In [1]:
import numpy as np
import pandas as pd
from os.path import join

In [2]:
# Read data
input_folder = "input_data"
output_folder = "output"
plastic_file = "world_plastic_trade.csv"
file_to = "AllTradeTo3Countries.csv"
file_from = "AllTradeFrom3Countries.csv"
input_to = pd.read_csv(join(input_folder, file_to))
input_from = pd.read_csv(join(input_folder, file_from))
plastic_data = pd.read_csv(join(input_folder, plastic_file))

Intial work indicated that data from 2018 seems to be more balanced than data from 2019 in terms of having both sides of the import and export by countries that should be in a reciprocal relationship. We will use that henceforth. 2020 is presumably anomalous. 

In [3]:
year = 2018
countries = ["South Africa", "Nigeria", "Egypt"]
impex = ["Import", "Export"]
plastic_waste = ["Waste, parings and scrap, of plastics", "Waste, parings and scrap, of plastics."]
report_cols = ["Reporter", "Trade Flow", "Netweight (kg)", "Infilledkg"]
composite = ["Coal; briquettes, ovoids and similar solid fuels manufactured from coal"]
crude = "Petroleum oils and oils obtained from bituminous minerals; crude"
redundant_cols = [
    "Classification", "Period", "Period Desc.", "Reporter Code", 
    "FOB Trade Value (US$)", "CIF Trade Value (US$)", 'Mode of Transport Code',
    "Alt Qty", "Alt Qty Unit Code", "Alt Qty Unit", 'Qty Unit Code', 
    "Customs", "Mode of Transport", "Customs Proc. Code", "Gross weight (kg)",
    "Qty Unit", "Qty"
]
from_year = input_from[input_from.Year==year][[col for col in input_from.columns if col not in redundant_cols]]
to_year = input_to[input_to.Year==year][[col for col in input_from.columns if col not in redundant_cols]]
plastic_year = plastic_data[plastic_data.Year==year][[col for col in plastic_data if col not in redundant_cols]]
assert set(from_year.columns) == set(to_year.columns)

In [4]:
plastic_year[["Reporter", "Trade Flow"]]

,Reporter,Trade Flow
217,Andorra,Import
218,Andorra,Export
219,Angola,Import
220,Angola,Export
221,Azerbaijan,Import
...,...,...
642,Sao Tome and Principe,Import
643,Sao Tome and Principe,Export
648,Barbados,Import
649,Barbados,Export


In [5]:
# Calculate the geometric mean, using only values that are within two orders of magnitude
def geomean_trunc(x):
    y = np.log(x[np.isfinite(x) & (x<100) & (x>0.01)])
    return np.exp(np.mean(y))
def geomean(x):
    y = np.log(x[(x != 0) & np.isfinite(x)])
    return np.exp(np.mean(y))
def logstdev_trunc(x):
    y = np.log(x[np.isfinite(x) & (x<100) & (x>0.01)])
    return np.exp(np.std(y))

In [6]:
plastic_year["ValueRatio"] = plastic_year["Trade Value (US$)"] / plastic_year["Netweight (kg)"]

In [7]:
plastic_year

,Year,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,2nd Partner Code,2nd Partner,2nd Partner ISO,Commodity Code,Commodity,Netweight (kg),Trade Value (US$),Flag,ValueRatio
217,2018,4,0,1,Import,Andorra,AND,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",5.0,21,0,4.200000
218,2018,4,0,2,Export,Andorra,AND,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",1061790.0,260004,0,0.244873
219,2018,4,0,1,Import,Angola,AGO,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",NaN,2155813,6,NaN
220,2018,4,0,2,Export,Angola,AGO,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",272855.0,133902,6,0.490744
221,2018,4,0,1,Import,Azerbaijan,AZE,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",4041242.0,1647736,0,0.407730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,2018,4,0,1,Import,Sao Tome and Principe,STP,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics.",1040.0,4222,0,4.059615
643,2018,4,0,2,Export,Sao Tome and Principe,STP,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics.",142.0,160,0,1.126761
648,2018,4,0,1,Import,Barbados,BRB,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics.",2066.0,11339,0,5.488383
649,2018,4,0,2,Export,Barbados,BRB,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics.",772446.0,134558,0,0.174197


We will infill the missing values in the table. We will use the geometric mean of the values in the table, since the values don't range over too large a range of values.

In [8]:
geomean(plastic_year["ValueRatio"][plastic_year["Commodity"].isin(plastic_waste)])

0.4149051392913076

In [9]:
geomean_trunc(plastic_year["ValueRatio"][plastic_year["Commodity"].isin(plastic_waste)])

0.4057896007508648

In [10]:
min(plastic_year["ValueRatio"][plastic_year["Commodity"].isin(plastic_waste)])

0.01425

In [11]:
ikg = "Infilledkg"
plastic_year[ikg] = plastic_year["Netweight (kg)"]
infill0 = ((plastic_year[ikg]==0) | np.isnan(plastic_year["Infilledkg"])) & (plastic_year["Commodity"].isin(plastic_waste))
plastic_ratio = geomean(plastic_year["ValueRatio"][plastic_year["Commodity"].isin(plastic_waste)])
plastic_year[ikg].loc[infill0] = plastic_year["Trade Value (US$)"]*plastic_ratio
plastic_year[plastic_year["Commodity"].isin(plastic_waste)]

c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Year,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,2nd Partner Code,2nd Partner,2nd Partner ISO,Commodity Code,Commodity,Netweight (kg),Trade Value (US$),Flag,ValueRatio,Infilledkg
217,2018,4,0,1,Import,Andorra,AND,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",5.0,21,0,4.200000,5.000000e+00
218,2018,4,0,2,Export,Andorra,AND,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",1061790.0,260004,0,0.244873,1.061790e+06
219,2018,4,0,1,Import,Angola,AGO,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",NaN,2155813,6,NaN,8.944579e+05
220,2018,4,0,2,Export,Angola,AGO,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",272855.0,133902,6,0.490744,2.728550e+05
221,2018,4,0,1,Import,Azerbaijan,AZE,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",4041242.0,1647736,0,0.407730,4.041242e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,2018,4,0,1,Import,Sao Tome and Principe,STP,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics.",1040.0,4222,0,4.059615,1.040000e+03
643,2018,4,0,2,Export,Sao Tome and Principe,STP,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics.",142.0,160,0,1.126761,1.420000e+02
648,2018,4,0,1,Import,Barbados,BRB,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics.",2066.0,11339,0,5.488383,2.066000e+03
649,2018,4,0,2,Export,Barbados,BRB,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics.",772446.0,134558,0,0.174197,7.724460e+05


In [12]:
big_plastic_report = plastic_year[
    plastic_year["Trade Flow"].isin(impex) & plastic_year.Commodity.isin(plastic_waste)
][report_cols]
big_plastic_report_recon = big_plastic_report.pivot(columns="Trade Flow", index=["Reporter"], values=ikg).copy()

In [13]:
big_plastic_report = big_plastic_report.pivot(columns="Trade Flow", index=["Reporter"], values="Netweight (kg)")
big_plastic_report["Net Import"] = big_plastic_report.Import - big_plastic_report.Export
big_plastic_report = big_plastic_report.sort_values(by="Import", ascending=False)

In [14]:
big_plastic_report_recon.head(10)

Trade Flow,Export,Import
Reporter,,
Andorra,1061790.0,5.000000e+00
Angola,272855.0,8.944579e+05
Antigua and Barbuda,16081.0,NaN
Argentina,NaN,3.901000e+04
Armenia,169382.0,4.000400e+04
Aruba,NaN,1.000000e+00
Australia,153831432.0,1.325389e+07
Austria,177370629.0,1.989346e+08
Azerbaijan,19000.0,4.041242e+06


In [15]:
big_plastic_report.head(10)

Trade Flow,Export,Import,Net Import
Reporter,,,
Malaysia,4.611468e+07,872530652.0,826415967.0
"China, Hong Kong SAR",2.877646e+08,598046363.0,310281733.0
Netherlands,3.417910e+08,555419225.0,213628198.0
Thailand,1.906198e+08,552726632.0,362106799.0
Germany,1.048687e+09,468022146.0,-580665148.0
USA,0.000000e+00,442291077.0,442291077.0
Turkey,1.564301e+07,436909935.0,421266924.0
"Other Asia, nes",6.082400e+07,429192961.0,368368959.0
Indonesia,9.851637e+07,320451809.0,221935441.0


In [16]:
big_plastic_report_recon["Net Import"] = big_plastic_report_recon.Import - big_plastic_report_recon.Export
big_plastic_report_recon = big_plastic_report_recon.sort_values(by="Import", ascending=False)
big_plastic_report_recon.head(10)

Trade Flow,Export,Import,Net Import
Reporter,,,
Malaysia,4.611468e+07,872530652.0,8.264160e+08
"China, Hong Kong SAR",2.877646e+08,598046363.0,3.102817e+08
Netherlands,3.417910e+08,555419225.0,2.136282e+08
Thailand,1.906198e+08,552726632.0,3.621068e+08
Germany,1.048687e+09,468022146.0,-5.806651e+08
USA,1.852439e+08,442291077.0,2.570472e+08
Turkey,1.564301e+07,436909935.0,4.212669e+08
"Other Asia, nes",6.082400e+07,429192961.0,3.683690e+08
Indonesia,9.851637e+07,320451809.0,2.219354e+08


In [17]:
plastic_report = big_plastic_report[big_plastic_report.index.isin(countries)]
plastic_report_reconstructed = big_plastic_report_recon[big_plastic_report_recon.index.isin(countries)]
plastic_report

Trade Flow,Export,Import,Net Import
Reporter,,,
Egypt,2999.0,19889853.0,19886854.0
South Africa,11389890.0,18074903.0,6685013.0
Nigeria,2164750.0,16612038.0,14447288.0


In [18]:
plastic_report_reconstructed = big_plastic_report_recon[big_plastic_report_recon.index.isin(countries)]
plastic_report_reconstructed

Trade Flow,Export,Import,Net Import
Reporter,,,
Egypt,2999.0,19889853.0,19886854.0
South Africa,11389890.0,18074903.0,6685013.0
Nigeria,2164750.0,16612038.0,14447288.0


We can compare these to the data found in https://www.statista.com/chart/14383/countries-importing-plastic-waste/
Unfortunately these are not even order-of-magnitude consistent, although the approximate ordering is right. 

# Investigate fossil fuels

In [19]:
from_year[from_year.Commodity==crude]

,Year,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,2nd Partner Code,2nd Partner,2nd Partner ISO,Commodity Code,Commodity,Netweight (kg),Trade Value (US$),Flag
542,2018,4,0,1,Import,South Africa,ZAF,0,World,WLD,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,2.027587e+10,10965006206,0
543,2018,4,0,2,Export,South Africa,ZAF,0,World,WLD,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,0.000000e+00,168652,6
544,2018,4,0,3,Re-Export,South Africa,ZAF,0,World,WLD,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,0.000000e+00,28601,0
545,2018,4,0,1,Import,South Africa,ZAF,24,Angola,AGO,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,2.352291e+09,1245263357,0
546,2018,4,0,2,Export,South Africa,ZAF,56,Belgium,BEL,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,6.000000e+00,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,2018,4,0,1,Import,Egypt,EGY,682,Saudi Arabia,SAU,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,8.197000e+08,408613217,0
1473,2018,4,0,2,Export,Egypt,EGY,699,India,IND,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,1.296000e+09,597663983,0
1474,2018,4,0,2,Export,Egypt,EGY,757,Switzerland,CHE,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,1.310000e+08,66417552,0
1475,2018,4,0,2,Export,Egypt,EGY,826,United Kingdom,GBR,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,4.020000e+08,192357516,0


In [20]:
# Correct for the values of imports with known financial value but unknown weight
nw = "Netweight (kg)"
tv = "Trade Value (US$)"
commodity_ratio = pd.DataFrame(columns=("Commodity", "ImpExp", "FromTo", "Value"))
for commodity in from_year.Commodity.unique():
    exp_from_ind = ((from_year["Trade Flow"] == "Export") & (from_year.Commodity == commodity))
    exp_to_ind = (to_year["Trade Flow"] == "Export") & (to_year.Commodity == commodity)
    imp_from_ind = (from_year["Trade Flow"] == "Import") & (from_year.Commodity == commodity)
    imp_to_ind = (to_year["Trade Flow"] == "Import") & (to_year.Commodity == commodity)
    
    fromrat = geomean(
        from_year[from_year.Commodity == commodity]["Trade Value (US$)"] / from_year[from_year.Commodity == commodity][nw]
    )
    torat = geomean(to_year[to_year.Commodity == commodity]["Trade Value (US$)"] / to_year[to_year.Commodity == commodity][nw])
    commodity_ratio = commodity_ratio.append(pd.DataFrame({
        "Commodity": [commodity, commodity, commodity, commodity, commodity, commodity],
        "ImpExp": ["Export", "Export", "Import", "Import", "Both", "Both"],
        "FromTo": ["From", "To", "From", "To", "From", "To"],
        "Value": [
            geomean(from_year[exp_from_ind]["Trade Value (US$)"] / from_year[exp_from_ind][nw]),
            geomean(to_year[exp_to_ind]["Trade Value (US$)"] / to_year[exp_to_ind][nw]),
            geomean(from_year[imp_from_ind]["Trade Value (US$)"] / from_year[imp_from_ind][nw]),
            geomean(to_year[imp_to_ind]["Trade Value (US$)"] / to_year[imp_to_ind][nw]),
            fromrat,
            torat
        ]
    }))
    infillindfrom = (from_year.Commodity == commodity) & ((from_year[nw] == 0) | np.isnan(from_year[nw]))
    infillindto = (to_year.Commodity == commodity) & ((to_year[nw] == 0) | np.isnan(to_year[nw]))
    from_year[nw].loc[infillindfrom] = from_year[tv][infillindfrom] * fromrat
    to_year[nw].loc[infillindto] = to_year[tv][infillindto] * torat

c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

In [21]:
commodity_ratio.head(15)

,Commodity,ImpExp,FromTo,Value
0,"Coal; briquettes, ovoids and similar solid fue...",Export,From,0.156749
1,"Coal; briquettes, ovoids and similar solid fue...",Export,To,0.252843
2,"Coal; briquettes, ovoids and similar solid fue...",Import,From,0.325354
3,"Coal; briquettes, ovoids and similar solid fue...",Import,To,0.219046
4,"Coal; briquettes, ovoids and similar solid fue...",Both,From,0.196584
5,"Coal; briquettes, ovoids and similar solid fue...",Both,To,0.230981
0,"Coal; anthracite, whether or not pulverised, b...",Export,From,0.285530
1,"Coal; anthracite, whether or not pulverised, b...",Export,To,0.265878
2,"Coal; anthracite, whether or not pulverised, b...",Import,From,0.427715
3,"Coal; anthracite, whether or not pulverised, b...",Import,To,0.362666


In [22]:
# To reconstruct weights in the database we will use the the average of the from values
commodity_ratio[commodity_ratio.FromTo=="From"].groupby("Commodity").mean()

,Value
Commodity,
"Bitumen and asphalt, natural; bituminous or oil shale and tar sands; asphaltites and asphaltic rocks",1.297741
"Bituminous mixtures based on natural asphalt; on natural bitumen, on petroleum bitumen, on mineral tar or on mineral tar pitch (e.g. bituminous mastics, cut-backs)",0.963755
"Briquettes, ovoids and similar solid fuels; manufactured from coal",0.338141
"Coal gas, water gas, producer gas and similar gases, other than petroleum gases and other gaseous hydrocarbons",6.857978
"Coal; (other than anthracite and bituminous), whether or not pulverised but not agglomerated",0.342600
"Coal; anthracite, whether or not pulverised, but not agglomerated",0.346416
"Coal; bituminous, whether or not pulverised, but not agglomerated",0.121035
"Coal; briquettes, ovoids and similar solid fuels manufactured from coal",0.226229
"Coke and semi-coke; of coal, lignite or peat, whether or not agglomerated; retort carbon",0.440768


In [23]:
world_sum_exp = pd.DataFrame(columns=("Country","Commodity", "ExportsFrom", "World", "ImportsTo", "ReExportsFrom"))
world_sum_imp = pd.DataFrame(columns=("Country","Commodity", "ImportsFrom", "World", "ReImportsFrom", "ExportsTo"))
for commodity in from_year.Commodity.unique():
    for country in countries:
        exports_from = from_year[(from_year["Trade Flow"] == "Export") & (from_year.Commodity == commodity) & (from_year.Reporter == country)]
        reexports_from = from_year[(from_year["Trade Flow"] == "Re-Export") & (from_year.Commodity == commodity) & (from_year.Reporter == country)]
        reimports_from = from_year[(from_year["Trade Flow"] == "Re-Import") & (from_year.Commodity == commodity) & (from_year.Reporter == country)]
        imports_to = to_year[(to_year["Trade Flow"] == "Import") & (to_year.Commodity == commodity) & (to_year.Partner == country)]
        exports_to = to_year[(to_year["Trade Flow"] == "Export") & (to_year.Commodity == commodity) & (to_year.Partner == country)]
        imports_from = from_year[(from_year["Trade Flow"] == "Import") & (from_year.Commodity == commodity) & (from_year.Reporter == country)]
        assert len(exports_from[exports_from.Partner=="World"][nw]) in [0, 1]
        world_sum_exp = world_sum_exp.append(pd.DataFrame({
            "Country": country,
            "Commodity": commodity, 
            "ExportsFrom": np.nansum(exports_from[exports_from.Partner!="World"][nw]),
            "World": exports_from[exports_from.Partner=="World"][nw],
            "ImportsTo": np.nansum(imports_to[imports_to.Partner!="World"][nw]), 
            "ReExportsFrom": np.nansum(reexports_from[reexports_from.Partner!="World"][nw]) 
        }))
        assert len(imports_from[imports_from.Partner=="World"][nw]) in [0, 1]
        world_sum_imp = world_sum_imp.append(pd.DataFrame({
            "Country": country,
            "Commodity": commodity, 
            "ImportsFrom": np.nansum(imports_from[imports_from.Partner!="World"][nw]), 
            "ReImportsFrom": np.nansum(reimports_from[reimports_from.Partner!="World"][nw]), 
            "World": np.nansum(imports_from[imports_from.Partner=="World"][nw]),
            "ExportsTo": np.nansum(exports_to[exports_to.Partner!="World"][nw])
        }, index=[1]))

In [24]:
# We need to avoid a zero division error
world_sum_imp["ImpFromExpToRatio"] = world_sum_imp.ImportsFrom / (world_sum_imp.ExportsTo + 1e-20)

In [25]:
world_sum_imp

,Country,Commodity,ImportsFrom,World,ReImportsFrom,ExportsTo,ImpFromExpToRatio
1,South Africa,"Coal; briquettes, ovoids and similar solid fue...",3.051784e+09,3.051784e+09,300727000.0,1.722565e+09,1.771651e+00
1,Nigeria,"Coal; briquettes, ovoids and similar solid fue...",5.929302e+07,1.441496e+06,0.0,9.372461e+07,6.326302e-01
1,Egypt,"Coal; briquettes, ovoids and similar solid fue...",1.932948e+09,6.885548e+07,0.0,5.597358e+09,3.453322e-01
1,South Africa,"Coal; anthracite, whether or not pulverised, b...",3.542647e+08,3.542647e+08,0.0,3.681602e+07,9.622568e+00
1,Nigeria,"Coal; anthracite, whether or not pulverised, b...",1.887503e+06,1.887503e+06,0.0,1.687869e+06,1.118276e+00
1,Egypt,"Coal; anthracite, whether or not pulverised, b...",9.974059e+07,8.673524e+06,0.0,2.122845e+07,4.698440e+00
1,South Africa,"Coal; bituminous, whether or not pulverised, b...",1.273978e+09,1.273978e+09,0.0,1.470915e+09,8.661127e-01
1,Nigeria,"Coal; bituminous, whether or not pulverised, b...",0.000000e+00,0.000000e+00,0.0,9.203600e+07,0.000000e+00
1,Egypt,"Coal; bituminous, whether or not pulverised, b...",1.447283e+09,2.434234e+07,0.0,5.545267e+09,2.609943e-01
1,South Africa,"Coal; (other than anthracite and bituminous), ...",1.379110e+09,1.379110e+09,300727000.0,1.555384e+08,8.866680e+00


In [26]:
world_sum_exp["ExpFromImpToRatio"] = world_sum_exp.ExportsFrom / world_sum_exp.ImportsTo

In [27]:
world_sum_exp

,Country,Commodity,ExportsFrom,World,ImportsTo,ReExportsFrom,ExpFromImpToRatio
1,South Africa,"Coal; briquettes, ovoids and similar solid fue...",8.131101e+10,8.131101e+10,6.574748e+10,2.059060e+05,1.236717e+00
1186,Nigeria,"Coal; briquettes, ovoids and similar solid fue...",2.682000e+05,2.682000e+05,3.286232e+05,0.000000e+00,8.161324e-01
1194,Egypt,"Coal; briquettes, ovoids and similar solid fue...",1.205169e+08,1.205169e+08,1.162292e+08,0.000000e+00,1.036890e+00
121,South Africa,"Coal; anthracite, whether or not pulverised, b...",1.240604e+09,1.240604e+09,1.081157e+09,2.005220e+05,1.147478e+00
1232,Egypt,"Coal; anthracite, whether or not pulverised, b...",1.203255e+08,1.203255e+08,1.158026e+08,0.000000e+00,1.039057e+00
174,South Africa,"Coal; bituminous, whether or not pulverised, b...",7.892341e+10,7.892341e+10,2.587525e+10,5.384000e+03,3.050150e+00
1259,Egypt,"Coal; bituminous, whether or not pulverised, b...",3.000000e+03,3.000000e+03,1.671700e+05,0.000000e+00,1.794580e-02
242,South Africa,"Coal; (other than anthracite and bituminous), ...",1.016501e+09,1.016501e+09,3.865670e+10,0.000000e+00,2.629561e-02
1266,Nigeria,"Coal; (other than anthracite and bituminous), ...",2.682000e+05,2.682000e+05,2.019579e+05,0.000000e+00,1.327999e+00
1272,Egypt,"Coal; (other than anthracite and bituminous), ...",1.675510e+05,1.675510e+05,2.590240e+05,0.000000e+00,6.468551e-01


In [28]:
# Calculate the means to establish re-imports/re-exports are only small fractions of the total
np.mean(world_sum_exp.ReExportsFrom / world_sum_exp.ExportsFrom)

0.017966631751115226

In [29]:
np.mean(world_sum_imp.ReImportsFrom / world_sum_imp.ImportsFrom)

0.0059875637886556594

In [30]:
geomean(world_sum_exp.ExpFromImpToRatio)

0.7583342656499067

In [31]:
geomean_trunc(world_sum_exp.ExpFromImpToRatio)

0.5766135232588026

In [32]:
logstdev_trunc(world_sum_exp.ExpFromImpToRatio)

5.109011903335072

In [33]:
geomean(world_sum_imp.ImpFromExpToRatio)

33.369109633771956

In [34]:
geomean_trunc(world_sum_imp.ImpFromExpToRatio)

1.4597896258929606

In [35]:
logstdev_trunc(world_sum_imp.ImpFromExpToRatio)

4.88571834123329

In [36]:
# Calculate the fraction of data within 2 orders: 
fraction_within_2_orders_exp = sum(
    (world_sum_exp.ExpFromImpToRatio < 100 ) & (world_sum_exp.ExpFromImpToRatio > 0.01)
)/len(world_sum_exp)
fraction_within_2_orders_exp

0.9047619047619048

In [37]:
fraction_within_2_orders_imp = sum(
    (world_sum_imp.ImpFromExpToRatio < 100 ) & (world_sum_imp.ImpFromExpToRatio > 0.01)
)/len(world_sum_imp)
fraction_within_2_orders_imp

0.8596491228070176

In [38]:
(fraction_within_2_orders_exp * len(world_sum_exp)  + fraction_within_2_orders_imp* len(world_sum_imp))/(len(world_sum_exp)+len(world_sum_imp))

0.8787878787878788

# Begin conversion into emissions equivalents
We use data from https://ghgprotocol.org/calculation-tools#cross_sector_tools_id to perform the conversion. This is in turn based on http://www.ipcc-nggip.iges.or.jp/public/2006gl/vol2.html data, but converted into units of emissions/tonne rather than emissions/TJ. We compare these numbers when applied to coal with the numbers using the USA data, found in https://www.epa.gov/sites/production/files/2020-04/documents/ghg-emission-factors-hub.pdf
to ensure that the values are reasonable. 
## USA analysis

In [39]:
us_folder = "USA_data"
solid_factor_file = "solidFuelsConversionFactors.csv"
solid_factors = pd.read_csv(join(input_folder, us_folder, solid_factor_file), index_col=0)
kgInShortTon = 907.18
solid_factors = solid_factors.append(pd.DataFrame({c: np.nan for c in solid_factors.columns}, index=["nan"]))
solid_factors

,mmBtu per short ton,kg CO2 per mmBtu,g CH4 per mmBtu,g N2O per mmBtu,kg CO2 per short ton,g CH4 per short ton,g N2O per short ton
Anthracite Coal,25.09,103.69,11.0,1.6,2602.0,276.0,40.0
Bituminous Coal,24.93,93.28,11.0,1.6,2325.0,274.0,40.0
Sub-bituminous Coal,17.25,97.17,11.0,1.6,1676.0,190.0,28.0
Lignite Coal,14.21,97.72,11.0,1.6,1389.0,156.0,23.0
Mixed Coal (Commercial Sector),21.39,94.27,11.0,1.6,2016.0,235.0,34.0
Mixed Coal (Electric Power Sector),19.73,95.52,11.0,1.6,1885.0,217.0,32.0
Mixed Coal (Industrial Coking),26.28,93.90,11.0,1.6,2468.0,289.0,42.0
Mixed Coal (Industrial Sector),22.35,94.67,11.0,1.6,2116.0,246.0,36.0
Coal Coke,24.80,113.67,11.0,1.6,2819.0,273.0,40.0
Municipal Solid Waste,9.95,90.70,32.0,4.2,902.0,318.0,42.0


In [40]:
# Work out how these names correspond to the comtrade names
input_from.Commodity.unique()

array(['Coal; briquettes, ovoids and similar solid fuels manufactured from coal',
       'Coal; anthracite, whether or not pulverised, but not agglomerated',
       'Coal; bituminous, whether or not pulverised, but not agglomerated',
       'Coal; (other than anthracite and bituminous), whether or not pulverised but not agglomerated',
       'Briquettes, ovoids and similar solid fuels; manufactured from coal',
       'Lignite; whether or not agglomerated, excluding jet',
       'Peat; (including peat litter), whether or not agglomerated',
       'Coke and semi-coke; of coal, lignite or peat, whether or not agglomerated; retort carbon',
       'Coal gas, water gas, producer gas and similar gases, other than petroleum gases and other gaseous hydrocarbons',
       'Tar distilled from coal, from lignite, peat and other mineral tars, whether or not dehydrated or partially distilled; including reconstituted tars',
       'Oils and other products of the distillation of high temperature coal t

In [41]:
solid_name_conversion = {
    'Coal; anthracite, whether or not pulverised, but not agglomerated': "Anthracite Coal", 
    'Coal; bituminous, whether or not pulverised, but not agglomerated': "Bituminous Coal",
    'Coal; (other than anthracite and bituminous), whether or not pulverised but not agglomerated': "Sub-bituminous Coal",
    'Briquettes, ovoids and similar solid fuels; manufactured from coal': "Mixed Coal (Commercial Sector)",
    'Lignite; whether or not agglomerated, excluding jet': "Lignite Coal",
    'Peat; (including peat litter), whether or not agglomerated': "Peat",
    'Coke and semi-coke; of coal, lignite or peat, whether or not agglomerated; retort carbon': "Coal Coke",
}
world_sum_exp = world_sum_exp
world_sum_exp["emissionsCommodity"] = [
    solid_name_conversion[v]  if v in solid_name_conversion.keys() else "nan" for v in world_sum_exp.Commodity
]
world_sum_imp["emissionsCommodity"] = [
    solid_name_conversion[v]  if v in solid_name_conversion.keys() else "nan" for v in world_sum_imp.Commodity
]
# Apply emissions conversion factors, including converting kg CO2 to tonnes Co2
world_sum_exp["tCO2_USA"] = solid_factors[
    "kg CO2 per short ton"
][world_sum_exp["emissionsCommodity"]].values * world_sum_exp.ExportsFrom / kgInShortTon / 1000 
world_sum_exp["tCH4_USA"] = solid_factors[
    "g CH4 per short ton"
][world_sum_exp["emissionsCommodity"]].values * world_sum_exp.ExportsFrom / kgInShortTon / 1000000
world_sum_exp["tN2O_USA"] = solid_factors[
    "g N2O per short ton"
][world_sum_exp["emissionsCommodity"]].values * world_sum_exp.ExportsFrom / kgInShortTon / 1000000
# And to imports
world_sum_imp["tCO2_USA"] = solid_factors[
    "kg CO2 per short ton"
][world_sum_imp["emissionsCommodity"]].values * world_sum_imp.ImportsFrom / kgInShortTon / 1000 
world_sum_imp["tCH4_USA"] = solid_factors[
    "g CH4 per short ton"
][world_sum_imp["emissionsCommodity"]].values * world_sum_imp.ImportsFrom / kgInShortTon / 1000000
world_sum_imp["tN2O_USA"] = solid_factors[
    "g N2O per short ton"
][world_sum_imp["emissionsCommodity"]].values * world_sum_imp.ImportsFrom / kgInShortTon / 1000000

## GHG protocol analysis

In [42]:
ghgprotocol_folder = "ghg_protocol_UN_data"
co2_file = "CO2_emissions_per_mass_ghgprotocol.csv"
ch4_file = "CH4_emissions_per_mass_ghgprotocol.csv"
n2o_file = "N2O_emissions_per_mass_ghgprotocol.csv"
co2_factors = pd.read_csv(join(input_folder, ghgprotocol_folder, co2_file), index_col=0)
ch4_factors = pd.read_csv(join(input_folder, ghgprotocol_folder, ch4_file), index_col=0)
n2o_factors = pd.read_csv(join(input_folder, ghgprotocol_folder, n2o_file), index_col=0)
co2_factors=co2_factors.append(
    pd.DataFrame({c: 0 for c in co2_factors.columns}, index=["Unburned"])
)
ch4_factors=ch4_factors.append(
    pd.DataFrame({c: 0 for c in ch4_factors.columns}, index=["Unburned"])
)
n2o_factors=n2o_factors.append(
    pd.DataFrame({c: 0 for c in n2o_factors.columns}, index=["Unburned"])
)
ch4_factors

,TJ/Gg,kg/TJ,kg/tonne
Crude oil,42.30,10,0.42300
Orimulsion,27.50,10,0.27500
Natural Gas Liquids,44.20,10,0.44200
Motor gasoline,44.30,10,0.44300
Aviation gasoline,44.30,10,0.44300
Jet gasoline,44.30,10,0.44300
Jet kerosene,44.10,10,0.44100
Other kerosene,43.80,10,0.43800
Shale oil,38.10,10,0.38100
Gas/Diesel oil,43.00,10,0.43000


In [43]:
world_sum_exp.Commodity.unique()

array(['Coal; briquettes, ovoids and similar solid fuels manufactured from coal',
       'Coal; anthracite, whether or not pulverised, but not agglomerated',
       'Coal; bituminous, whether or not pulverised, but not agglomerated',
       'Coal; (other than anthracite and bituminous), whether or not pulverised but not agglomerated',
       'Briquettes, ovoids and similar solid fuels; manufactured from coal',
       'Lignite; whether or not agglomerated, excluding jet',
       'Peat; (including peat litter), whether or not agglomerated',
       'Coke and semi-coke; of coal, lignite or peat, whether or not agglomerated; retort carbon',
       'Coal gas, water gas, producer gas and similar gases, other than petroleum gases and other gaseous hydrocarbons',
       'Tar distilled from coal, from lignite, peat and other mineral tars, whether or not dehydrated or partially distilled; including reconstituted tars',
       'Oils and other products of the distillation of high temperature coal t

In [44]:
ghg_factors_conversion = {
    'Coal; anthracite, whether or not pulverised, but not agglomerated': "Anthracite",
    'Coal; bituminous, whether or not pulverised, but not agglomerated': "Bitumen",
    'Coal; (other than anthracite and bituminous), whether or not pulverised but not agglomerated': "Sub bituminous coal",
    'Briquettes, ovoids and similar solid fuels; manufactured from coal': "Other bituminous coal",
    'Lignite; whether or not agglomerated, excluding jet': "Lignite",
    'Peat; (including peat litter), whether or not agglomerated': "Peat",
    'Coke and semi-coke; of coal, lignite or peat, whether or not agglomerated; retort carbon': "Coking coal",
    'Coal gas, water gas, producer gas and similar gases, other than petroleum gases and other gaseous hydrocarbons': "Gas coke",
    'Tar distilled from coal, from lignite, peat and other mineral tars, whether or not dehydrated '
    'or partially distilled; including reconstituted tars': "Coal tar",
    'Oils and other products of the distillation of high temperature coal tar; similar products in which the weight '
    'of the aromatic constituents exceeds that of the non-aromatic constituents': "Other petroleum products",
    'Pitch and pitch coke; obtained from coal tar or from other mineral tars': "Coke oven coke",
    'Petroleum oils and oils obtained from bituminous minerals; crude': "Crude oil",
    'Petroleum oils and oils from bituminous minerals, not crude; preparations n.e.c, containing by weight 70% '
    'or more of petroleum oils or oils from bituminous minerals; these being the basic constituents of '
    'the preparations; waste oils': "Motor gasoline",
    'Petroleum gases and other gaseous hydrocarbons': "Refinery gas",
    'Petroleum jelly; paraffin wax, micro-crystalline petroleum wax, slack wax, ozokerite, lignite wax, '
    'peat wax, other mineral waxes, similar products obtained by synthesis, other processes; coloured or not': "Unburned",
    'Petroleum coke, petroleum bitumen; other residues of petroleum oils or oils obtained from '
    'bituminous minerals': "Petroleum coke",
    'Bitumen and asphalt, natural; bituminous or oil shale and tar sands; asphaltites '
    'and asphaltic rocks': "Oil shale and tar sands",
    'Bituminous mixtures based on natural asphalt; on natural bitumen, on petroleum bitumen, on mineral '
    'tar or on mineral tar pitch (e.g. bituminous mastics, cut-backs)': "Unburned"
}

In [45]:
world_sum_exp["ghgCategory"] = [
    ghg_factors_conversion[v]  if v in ghg_factors_conversion.keys() else "nan" for v in world_sum_exp.Commodity
]
world_sum_imp["ghgCategory"] = [
    ghg_factors_conversion[v]  if v in ghg_factors_conversion.keys() else "nan" for v in world_sum_imp.Commodity
]
# We want to remove the aggregate values to prevent confusion:
world_sum_exp = world_sum_exp[~world_sum_exp.Commodity.isin(composite)]
world_sum_imp = world_sum_imp[~world_sum_imp.Commodity.isin(composite)]

In [46]:
# Convert from kg product to tonnes, then kg CO2 to tonnes
world_sum_exp["tCO2"] = world_sum_exp.ExportsFrom * co2_factors["kg/tonne"][world_sum_exp.ghgCategory].values / 1000000
world_sum_exp["tCH4"] = world_sum_exp.ExportsFrom * ch4_factors["kg/tonne"][world_sum_exp.ghgCategory].values / 1000000
world_sum_exp["tN2O"] = world_sum_exp.ExportsFrom * n2o_factors["kg/tonne"][world_sum_exp.ghgCategory].values / 1000000
world_sum_imp["tCO2"] = world_sum_imp.ImportsFrom * co2_factors["kg/tonne"][world_sum_imp.ghgCategory].values / 1000000
world_sum_imp["tCH4"] = world_sum_imp.ImportsFrom * ch4_factors["kg/tonne"][world_sum_imp.ghgCategory].values / 1000000
world_sum_imp["tN2O"] = world_sum_imp.ImportsFrom * n2o_factors["kg/tonne"][world_sum_imp.ghgCategory].values / 1000000
world_sum_imp

c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\ipykernel_launcher.py:4: Settin

,Country,Commodity,ImportsFrom,World,ReImportsFrom,ExportsTo,ImpFromExpToRatio,emissionsCommodity,tCO2_USA,tCH4_USA,tN2O_USA,ghgCategory,tCO2,tCH4,tN2O
1,South Africa,"Coal; anthracite, whether or not pulverised, b...",3.542647e+08,3.542647e+08,0.0,3.681602e+07,9.622568e+00,Anthracite Coal,1.016112e+06,107.781320,15.620481,Anthracite,9.298067e+05,94.588676,1.418830e+01
1,Nigeria,"Coal; anthracite, whether or not pulverised, b...",1.887503e+06,1.887503e+06,0.0,1.687869e+06,1.118276e+00,Anthracite Coal,5.413791e+03,0.574253,0.083225,Anthracite,4.953959e+03,0.503963,7.559450e-02
1,Egypt,"Coal; anthracite, whether or not pulverised, b...",9.974059e+07,8.673524e+06,0.0,2.122845e+07,4.698440e+00,Anthracite Coal,2.860788e+05,30.345027,4.397830,Anthracite,2.617801e+05,26.630736,3.994610e+00
1,South Africa,"Coal; bituminous, whether or not pulverised, b...",1.273978e+09,1.273978e+09,0.0,1.470915e+09,8.661127e-01,Bituminous Coal,3.265062e+06,384.785744,56.173101,Bitumen,4.132963e+06,512.139097,3.072835e+01
1,Nigeria,"Coal; bituminous, whether or not pulverised, b...",0.000000e+00,0.000000e+00,0.0,9.203600e+07,0.000000e+00,Bituminous Coal,0.000000e+00,0.000000,0.000000,Bitumen,0.000000e+00,0.000000,0.000000e+00
1,Egypt,"Coal; bituminous, whether or not pulverised, b...",1.447283e+09,2.434234e+07,0.0,5.545267e+09,2.609943e-01,Bituminous Coal,3.709223e+06,437.129893,63.814583,Bitumen,4.695188e+06,581.807700,3.490846e+01
1,South Africa,"Coal; (other than anthracite and bituminous), ...",1.379110e+09,1.379110e+09,300727000.0,1.555384e+08,8.866680e+00,Sub-bituminous Coal,2.547882e+06,288.841048,42.566049,Sub bituminous coal,2.504863e+06,260.651712,3.909776e+01
1,Nigeria,"Coal; (other than anthracite and bituminous), ...",2.120590e+06,2.120590e+06,0.0,0.000000e+00,2.120590e+26,Sub-bituminous Coal,3.917755e+03,0.444137,0.065452,Sub bituminous coal,3.851607e+03,0.400792,6.011873e-02
1,Egypt,"Coal; (other than anthracite and bituminous), ...",2.146517e+07,2.146517e+07,0.0,3.085194e+07,6.957480e-01,Sub-bituminous Coal,3.965655e+04,4.495671,0.662520,Sub bituminous coal,3.898698e+04,4.056918,6.085377e-01
1,South Africa,"Briquettes, ovoids and similar solid fuels; ma...",4.443221e+07,4.443221e+07,0.0,5.706020e+07,7.786901e-01,Mixed Coal (Commercial Sector),9.874042e+04,11.509920,1.665265,Other bituminous coal,1.084448e+05,11.463510,1.719527e+00


In [47]:
world_sum_exp.to_csv(join(output_folder, "detailed_exports.csv"))
report_exp = world_sum_exp.groupby(by="Country", axis=0).sum()
report_exp["GWP100 tCO2-equiv"] = report_exp.tCO2 + 28 * report_exp.tCH4 + 265 * report_exp.tN2O
report_exp.to_csv(join(output_folder, "total_exports.csv"))
report_exp

,ExportsFrom,World,ImportsTo,ReExportsFrom,ExpFromImpToRatio,tCO2_USA,tCH4_USA,tN2O_USA,tCO2,tCH4,tN2O,GWP100 tCO2-equiv
Country,,,,,,,,,,,,
Egypt,1.450403e+10,1.450403e+10,1.700420e+10,0.000000e+00,inf,8.393329e+05,87.196804,12.735905,4.365339e+07,5803.622692,334.000303,4.390440e+07
Nigeria,8.126515e+10,9.253793e+10,1.064272e+11,0.000000e+00,2.689025e+04,5.096705e+02,0.057545,0.008479,2.515155e+08,34081.971494,2028.277096,2.530073e+08
South Africa,8.746437e+10,8.589638e+10,6.947387e+10,8.139019e+06,2.404644e+01,2.083306e+08,24494.037676,3575.638653,2.797206e+08,34872.593692,2144.479805,2.812653e+08


In [48]:
world_sum_imp.to_csv(join(output_folder, "detailed_imports.csv"))
report_imp = world_sum_imp.groupby(by="Country", axis=0).sum()
report_imp["GWP100 tCO2-equiv"] = report_imp.tCO2 + 28 * report_imp.tCH4 + 265 * report_imp.tN2O
report_imp.to_csv(join(output_folder, "total_imports.csv"))
report_imp

,ImportsFrom,World,ReImportsFrom,ExportsTo,ImpFromExpToRatio,tCO2_USA,tCH4_USA,tN2O_USA,tCO2,tCH4,tN2O,GWP100 tCO2-equiv
Country,,,,,,,,,,,,
Egypt,2.740734e+10,2.271590e+10,0.0,1.670973e+10,7.090300e+29,6.156402e+06,679.343014,99.219523,8.223051e+07,10274.252500,597.235397,8.267646e+07
Nigeria,1.493154e+10,1.799315e+10,0.0,1.250540e+10,2.120590e+26,1.121559e+04,1.245889,0.181108,4.566571e+07,6556.107976,392.813482,4.595337e+07
South Africa,3.585824e+10,3.585824e+10,301247673.0,1.216273e+10,5.957142e+01,9.474703e+06,1050.158874,153.496368,1.077972e+08,14174.033168,873.191599,1.084254e+08


In [49]:
report_net_exp = report_exp - report_imp
report_net_exp = report_net_exp[["GWP100 tCO2-equiv", "tCO2", "tCH4", "tN2O"]]
report_net_exp.to_csv(join(output_folder, "net_exports.csv"))
report_net_exp

,GWP100 tCO2-equiv,tCO2,tCH4,tN2O
Country,,,,
Egypt,-3.877206e+07,-3.857712e+07,-4470.629808,-263.235094
Nigeria,2.070540e+08,2.058498e+08,27525.863518,1635.463614
South Africa,1.728399e+08,1.719234e+08,20698.560524,1271.288206


# Sanity check against Nigeria's oil exports

In [50]:
# Net exports were about 2mil barrels per day: https://ihsmarkit.com/research-analysis/nigerian-crude-oil-exports-to-approach-17-million-bd-in-2020.html
ReportedNigerianOilExport = 140*2000000*365
# And net imports is around 20 billion litres: https://www.statista.com/statistics/1165962/petrol-import-in-nigeria/
ReportedNigerianNetOilImport = 20000000000

In [51]:
ReportedNigerianOilExport

102200000000

In [52]:
ReportedNigerianNetOilImport/ReportedNigerianOilExport

0.19569471624266144

In [53]:
NigerianOilExp = world_sum_exp[(world_sum_exp.Country=="Nigeria") & (world_sum_exp.ghgCategory.isin(
    ["Crude oil", "Motor gasoline", "Other petroleum products"]
))]
NigerianOilExp

,Country,Commodity,ExportsFrom,World,ImportsTo,ReExportsFrom,ExpFromImpToRatio,emissionsCommodity,tCO2_USA,tCH4_USA,tN2O_USA,ghgCategory,tCO2,tCH4,tN2O
1375,Nigeria,Oils and other products of the distillation of...,1.703527e+08,1.703527e+08,6.335719e+03,0.0,26887.664616,nan,NaN,NaN,NaN,Other petroleum products,5.019715e+05,68.481785,4.108907
1424,Nigeria,Petroleum oils and oils obtained from bitumino...,7.913612e+10,7.913612e+10,8.936066e+10,0.0,0.885581,nan,NaN,NaN,NaN,Crude oil,2.453687e+08,33474.580193,2008.474812
1478,Nigeria,Petroleum oils and oils from bituminous minera...,2.769994e+08,2.769994e+08,1.386831e+09,0.0,0.199735,nan,NaN,NaN,NaN,Motor gasoline,8.503854e+05,122.710743,7.362645


In [54]:
sum(NigerianOilExp.ExportsFrom) / ReportedNigerianOilExport

0.7787032828543713

In [55]:
NigerianOilImp = world_sum_imp[(world_sum_imp.Country=="Nigeria") & (world_sum_imp.ghgCategory.isin(
    ["Crude oil", "Motor gasoline", "Other petroleum products"]
))]
NigerianOilImp

,Country,Commodity,ImportsFrom,World,ReImportsFrom,ExportsTo,ImpFromExpToRatio,emissionsCommodity,tCO2_USA,tCH4_USA,tN2O_USA,ghgCategory,tCO2,tCH4,tN2O
1,Nigeria,Oils and other products of the distillation of...,3.818150e+06,3.818151e+06,0.0,2.812205e+06,1.357707,nan,NaN,NaN,NaN,Other petroleum products,1.125079e+04,1.534896,0.092094
1,Nigeria,Petroleum oils and oils obtained from bitumino...,0.000000e+00,0.000000e+00,0.0,4.012510e+05,0.000000,nan,NaN,NaN,NaN,Crude oil,0.000000e+00,0.000000,0.000000
1,Nigeria,Petroleum oils and oils from bituminous minera...,1.463404e+10,1.779458e+10,0.0,1.208001e+10,1.211427,nan,NaN,NaN,NaN,Motor gasoline,4.492637e+07,6482.881518,388.972891


In [57]:
sum(NigerianOilImp.ImportsFrom) / ReportedNigerianNetOilImport

0.7318931104517388

The sanity check indicates that the difference of the imports and exports is more robust than either individually, possibly due to transport of oil through Nigeria